In [ ]:
# install the requirements
# !pip install flair

# `flair`: la librería NLP de Zalando Research

La compañia Zalando tiene necesidades de aplicar NLP en distintos ámbitos y su equipo de investigaación ha liberado recientemente [`flair`](https://github.com/zalandoresearch/flair), su librería de NLP.

`flair` permite acceder a funcionalidades muy interesantes para procesar lenguaje natural, algunas de ellas muy modernas como:

- [etiquetar morfo-sintácticamente](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_2_TAGGING.md)
- extraer entidades
- clasificar automáticamente texto
- entrenar tus propios modelos para [construir otros clasificadores](https://towardsdatascience.com/text-classification-with-state-of-the-art-nlp-library-flair-b541d7add21f)
- [cargar vectores de palabras en decenas de lenguas](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_3_WORD_EMBEDDING.md)
- [usar vectores contextuales como BERT, ELMo](https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_4_ELMO_BERT_FLAIR_EMBEDDING.md)

Veamos cómo podemos acceder a algunas de sus funcionalidades.

## Análisis morfo-sintáctico

Para analizar sintácticamente un texto, necesitamos cargar un etiquetador con un modelo concreto de información morfo-sintáctica. Por ejemplo, uno capaz de analizar varias lenguas.


In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# cargamos el analizador multi-idioma
tagger = SequenceTagger.load("pos-multi-fast")

In [ ]:
sentence1 = Sentence(
    "Facebook nació hace década y media tras una noche de copas de Mark Zuckerberg. "
)
tagger.predict(sentence1)
# imprimimos el análisis
print(sentence1.to_tagged_string())

sentence2 = Sentence(
    "Grand débat national: suivez Emmanuel Macron en direct de Bordeaux. "
)
tagger.predict(sentence2)
# imprimimos el análisis
print(sentence2.to_tagged_string())

sentence3 = Sentence(
    "Hier an der Zufahrt zur Startrampe 39A, wo vor 50 Jahren die gigantischen Saturn-Raketen der Apollo-Mondmissionen im Schneckentempo vorbeigefahren sind, prangen nun die blauen Lettern des Raumfahrtunternehmens von Elon Musk an einem Hangar."
)
tagger.predict(sentence3)
# imprimimos el análisis
print(sentence3.to_tagged_string())

## Reconocimiento de entidades

Para el reconocimiento de entidades varios modelos en diferentes lenguas. Aquí probamos con uno entrenado solo para inglés.

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# cargamos el reconocedor de entidades
tagger = SequenceTagger.load("ner-fast")

In [ ]:
# analizamos una oración
sentence = Sentence(
    "Behind closed doors, freshman Rep. Alexandria Ocasio-Cortez threatened to put those voting with Republicans “on a list” for a primary challenge in the 2020 election."
)
tagger.predict(sentence)

# imprimimos el análisis
print(sentence.to_tagged_string())

# iteramos por la entidades
for entity in sentence.get_spans("ner"):
    print(entity)

# o imprimimos la estructura de datos con el análisis completo
print(sentence.to_dict(tag_type="ner"))

In [ ]:
entity.tag

## Análisis de Opinión

También podemos utilizar un clasificador de textos y cargar el modelo entrenado con sentimiento, para poder detectar opiniones positivas y negativas.

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load("en-sentiment")

In [ ]:
sentence1 = Sentence("I love ice-cream!")
classifier.predict(sentence1)
print("La frase '{}' es {}".format(sentence1.to_plain_string(), sentence1.labels))

sentence2 = Sentence("Don't ever go to this restaurant. The food was horrible :-(")
classifier.predict(sentence2)
print("La frase '{}' es {}".format(sentence2.to_plain_string(), sentence2.labels))